In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

os.environ["USE_PYGEOS"] = "0"
import geopandas as gpd

sys.path.append("..")
sys.path.append("../..")
from src.paper_analysis_hydropower import (
    NationalAnalysisHydropower,
    plot_pre_post_bias_correction_validation,
)

In [ ]:
cm = 1 / 2.54

In [ ]:
paths = json.load(open("../paths.json"))

path_data = Path(paths["path_data"])

path_data_hydro = path_data / "hydropower"
path_data_polygons = path_data / "polygons"
path_swiss_maps = path_data / "maps" / "swissboundaries3d_2023-01_2056_5728/"

path_figs = Path(paths["path_figs"])

In [ ]:
gdf_switzerland = gpd.read_file(
    path_swiss_maps / "swissBOUNDARIES3D_1_4_TLM_LANDESGEBIET.shp"
).to_crs("EPSG:2056")

## Load BAFU polygons
gdf_polygons = gpd.read_file(path_data_polygons / "EZG_Gewaesser.gpkg")

## Load collocated hydropower locations and BAFU polygons
df_hydropower_polygons = pd.read_json(
    path_data_hydro / "hydropower_polygons" / "df_hydropower_polygons.json",
    orient="records",
)
gdf_hydropower_polygons = gpd.GeoDataFrame(
    df_hydropower_polygons,
    geometry=gpd.points_from_xy(
        df_hydropower_polygons["_x"], df_hydropower_polygons["_y"]
    ),
    crs="EPSG:2056",
)

## Load hydropower characteristics
df_wasta = pd.read_excel(
    path_data_hydro / "stats_hydropower_ch" / "wasta_2023_updated.xlsx"
)

## Load hydropower production (with simplified efficiency and with/without beta coefficient)
ds_hydropower_generation = xr.open_dataset(
    path_data_hydro
    / "hydropower_generation"
    / "ds_prevah_500_hydropower_production_ror_simplified_efficiency.nc"
)
ds_hydropower_generation_beta = xr.open_dataset(
    path_data_hydro
    / "hydropower_generation"
    / "ds_prevah_500_hydropower_production_ror_simplified_efficiency_with_beta.nc"
)
ds_hydropower_generation_merged = xr.merge(
    [
        ds_hydropower_generation.rename({"gen": "hp_no_beta"}),
        ds_hydropower_generation_beta.rename({"gen": "hp_with_beta"}),
    ]
).load()

df_hydropower_production_params = pd.read_csv(
    path_data_hydro
    / "hydropower_generation"
    / "ds_prevah_500_hydropower_production_parameters.csv"
)
df_hydropower_production_params.loc[
    df_hydropower_production_params["Expected yearly generation"] == 0,
    "Expected yearly generation",
] = df_hydropower_production_params[
    df_hydropower_production_params["Expected yearly generation"] == 0
].apply(lambda row: round(row["Capacity"] * 365 * 24 * 1e-6, 2), axis=1)

## Load historical Swiss electricity generation data
df_historical_data = pd.read_csv(
    path_data / "energy" / "ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv"
)

# Analyse data (without monthly bias correction)

In [ ]:
national_analysis_pre_correction = NationalAnalysisHydropower(
    gdf_switzerland,
    gdf_hydropower_polygons,
    df_wasta,
    ds_hydropower_generation_merged,
    df_hydropower_production_params,
    df_historical_data[
        [
            "Jahr",
            "Monat",
            "Erzeugung_laufwerk_GWh",
            "Erzeugung_speicherwerk_GWh",
            "Verbrauch_speicherpumpen_GWh",
        ]
    ],
    path_figs,
)

In [ ]:
national_analysis_pre_correction.aggregate_yearly_estimated_generation(
    with_percentage=False
)
national_analysis_pre_correction.aggregate_yearly_estimated_generation_with_operation_start(
    with_percentage=False
)
national_analysis_pre_correction.aggregate_yearly_estimated_generation_per_hp()
national_analysis_pre_correction.aggregate_yearly_estimated_generation_per_hp(
    reference_period=slice("1991", "2020")
)

In [ ]:
national_analysis_pre_correction.aggregate_seasonal_estimated_generation(
    with_operation_start=False
)
national_analysis_pre_correction.aggregate_seasonal_estimated_generation(
    with_operation_start=True
)
national_analysis_pre_correction.aggregate_seasonal_estimated_generation(
    with_operation_start=False, per_hydropower=True
)
national_analysis_pre_correction.aggregate_reference_seasonal_estimated_generation()

# Analyse data (with monthly bias correction)

In [ ]:
ds_monthly_bias_correction_factors = xr.open_dataset(
    path_data_hydro
    / "hydropower_generation"
    / "ds_prevah_500_hydropower_production_ror_simplified_efficiency_monthly_bias_correction_factors.nc"
).bias_correction_factor.load()

In [ ]:
national_analysis = NationalAnalysisHydropower(
    gdf_switzerland,
    gdf_hydropower_polygons,
    df_wasta,
    ds_hydropower_generation_merged * ds_monthly_bias_correction_factors,
    df_hydropower_production_params,
    df_historical_data[
        [
            "Jahr",
            "Monat",
            "Erzeugung_laufwerk_GWh",
            "Erzeugung_speicherwerk_GWh",
            "Verbrauch_speicherpumpen_GWh",
        ]
    ],
    path_figs,
)

In [ ]:
national_analysis.aggregate_yearly_estimated_generation(with_percentage=False)
national_analysis.aggregate_yearly_estimated_generation_with_operation_start(
    with_percentage=False
)
national_analysis.aggregate_yearly_estimated_generation_per_hp()
national_analysis.aggregate_yearly_estimated_generation_per_hp(
    reference_period=slice("1991", "2020")
)

In [ ]:
national_analysis.aggregate_seasonal_estimated_generation(with_operation_start=False)
national_analysis.aggregate_seasonal_estimated_generation(with_operation_start=True)
national_analysis.aggregate_seasonal_estimated_generation(
    with_operation_start=False, per_hydropower=True
)
national_analysis.aggregate_reference_seasonal_estimated_generation()

# Paper plots

## Fig 2

In [ ]:
plot_pre_post_bias_correction_validation(
    national_analysis_pre_correction,
    national_analysis,
    with_percentage=False,
    yearly_column_to_plot="Estimated Generation No Beta",
    winter_column_to_plot="Estimated Generation No Beta Winter",
    summer_column_to_plot="Estimated Generation No Beta Summer",
    subplots_titles=["Yearly", "Winter", "Summer"],
    start_yaxis_at_zero=True,
    save=True,
    output_filename="fig_2_bis.pdf",
    file_format="pdf",
)

## Fig 3

In [ ]:
national_analysis.plot_quantile_maps(
    yearly=True,
    variable_name="hp_no_beta",
    save=True,
    with_operation_start=True,
    with_decade_visualization=True,
    output_filename="fig_3.eps",
    file_format="eps",
)

In [ ]:
national_analysis.plot_quantile_maps(
    yearly=True,
    variable_name="hp_no_beta",
    nb_plots_rows=5,
    save=True,
    with_operation_start=True,
    with_decade_visualization=False,
    output_filename="fig_3_bis.eps",
    file_format="eps",
)

In [ ]:
national_analysis.plot_quantile_maps_selected_years(
    yearly=True,
    variable_name="hp_no_beta",
    save=True,
    output_filename="poster_fig_3.pdf",
    years=[1999, 2003, 2018, 2022],
    nb_cols=2,
    nb_rows=2,
)

## Fig 4

In [ ]:
national_analysis.plot_trend_analysis(
    with_percentage=False,
    yearly_column_to_plot="Estimated Generation No Beta",
    winter_column_to_plot="Estimated Generation No Beta Winter",
    summer_column_to_plot="Estimated Generation No Beta Summer",
    subplots_titles=["Yearly", "Winter", "Summer"],
    save=True,
    output_filename="fig_4_bis.pdf",
    file_format="pdf",
)

In [ ]:
ror_capacity_expansion_since_1991 = national_analysis.gdf_hydropower_locations.loc[
    (
        national_analysis.gdf_hydropower_locations["BeginningOfOperation"] > 1991
    ) & (
        national_analysis.gdf_hydropower_locations["Type"] == "L"
    )
]

print(
    f"{len(ror_capacity_expansion_since_1991)} RoR hydropower plants have been built since 1991"
)

print(
    f'''The capacity of RoR hydropower in Switzerland has increased from 1991 by {
        round(ror_capacity_expansion_since_1991['Capacity'].sum(), 2)
    } GW'''
)

## Metrics

### Bias

In [ ]:
national_analysis_pre_correction.compute_pred_bias()

In [ ]:
national_analysis_pre_correction.compute_pred_bias(percentage_bias=True)

In [ ]:
national_analysis_pre_correction.compute_pred_bias(yearly=False)

In [ ]:
national_analysis_pre_correction.compute_pred_bias(yearly=False, percentage_bias=True)

In [ ]:
national_analysis.compute_pred_bias()

In [ ]:
national_analysis.compute_pred_bias(yearly=False)

### Variability Metrics

#### Biased

In [ ]:
std_deviation_pre_correction_yearly = national_analysis_pre_correction.std_deviation(
    confidence_interval=True, confidence_level=0.95
)
std_deviation_pre_correction_yearly

In [ ]:
std_deviation_pre_correction_seasonal = national_analysis_pre_correction.std_deviation(
    yearly=False, confidence_interval=True, confidence_level=0.95
)
std_deviation_pre_correction_seasonal

#### Bias-corrected

In [ ]:
std_deviation_yearly = national_analysis.std_deviation(
    confidence_interval=True, confidence_level=0.95
)
std_deviation_yearly

In [ ]:
std_deviation_seasonal = national_analysis.std_deviation(
    yearly=False, confidence_interval=True, confidence_level=0.95
)
std_deviation_seasonal

In [ ]:
national_analysis.test_equality_of_variance(yearly=True)

### Trends in data

In [ ]:
df_hydropower_yearly = (
    national_analysis.create_dataframe_yearly_values(
        with_operation_start=True, with_percentage=False
    )
    .merge(
        national_analysis.create_dataframe_yearly_values(
            with_operation_start=False, with_percentage=False
        ),
        left_index=True,
        right_index=True,
        suffixes=("", "_fixed_system_2022"),
    )
    .merge(
        national_analysis.create_dataframe_yearly_values(
            with_operation_start=False,
            with_percentage=False,
            with_first_year_infrastructure=True,
        ),
        left_index=True,
        right_index=True,
        suffixes=("", "_fixed_system_1991"),
    )
)

In [ ]:
df_trends_yearly = national_analysis.compute_hydropower_generation_different_capacities_trend_slopes(
    "Estimated Generation No Beta", df_hydropower_yearly
)
df_trends_yearly

In [ ]:
-0.042 * 32, -0.088 * 32, 0.003 * 32

In [ ]:
trend_percentage_evolving_capacities = round((df_trends_yearly.set_index("name").loc["Evolving capacities", "coef"] / df_hydropower_yearly["Estimated Generation No Beta"].mean()) * 100, 3)
trend_percentage_1991_capacities = round((df_trends_yearly.set_index("name").loc["1991 capacities", "coef"] / df_hydropower_yearly["Estimated Generation No Beta_fixed_system_1991"].mean()) * 100, 3)

print("The trend of hydropower generation with evolving capacities is "
      f"{trend_percentage_evolving_capacities} % per year.")
print("The trend of hydropower generation with 1991 capacities is "
      f"{trend_percentage_1991_capacities} % per year.")

In [ ]:
# Total generation loss over time period 1991-2022 with fixed system 1991
len(df_hydropower_yearly) * df_trends_yearly.set_index("name").loc["1991 capacities", ["coef", "lower", "upper"]]

In [ ]:
(len(df_hydropower_yearly) * df_trends_yearly.set_index("name").loc["1991 capacities", ["coef", "lower", "upper"]] / df_hydropower_yearly["Estimated Generation No Beta_fixed_system_1991"].sum()) * 100

In [ ]:
df_hydropower_seasonal = (
    national_analysis.create_dataframe_seasonal_values(
        with_operation_start=True, with_percentage=False
    )
    .merge(
        national_analysis.create_dataframe_seasonal_values(
            with_operation_start=False, with_percentage=False
        ),
        left_index=True,
        right_index=True,
        suffixes=("", "_fixed_system_2022"),
    )
    .merge(
        national_analysis.create_dataframe_seasonal_values(
            with_operation_start=False,
            with_percentage=False,
            with_first_year_infrastructure=True,
        ),
        left_index=True,
        right_index=True,
        suffixes=("", "_fixed_system_1991"),
    )
)

In [ ]:
national_analysis.compute_hydropower_generation_different_capacities_trend_slopes(
    "Estimated Generation No Beta Summer", df_hydropower_seasonal
)

In [ ]:
national_analysis.compute_hydropower_generation_different_capacities_trend_slopes(
    "Estimated Generation No Beta Winter", df_hydropower_seasonal
)

#### Signal to noise ratio

In [ ]:
coef, _, _, conf_interval, residuals = national_analysis.compute_trend_statsmodel(
    np.arange(len(df_hydropower_yearly)).reshape(-1, 1),
    df_hydropower_yearly["Estimated Generation No Beta"].values,
)
snr = np.round(coef * len(df_hydropower_yearly) / np.std(residuals, ddof=1), 3)
print(f"SNR of the long term change in the time series with evolving capacities: {snr}")

In [ ]:
coef, _, _, conf_interval, residuals = national_analysis.compute_trend_statsmodel(
    np.arange(len(df_hydropower_yearly)).reshape(-1, 1),
    df_hydropower_yearly["Estimated Generation No Beta_fixed_system_1991"].values,
    alpha=0.05,
)
snr = np.round(coef * len(df_hydropower_yearly) / np.std(residuals, ddof=1), 3)
print(
    f"SNR of the long term change in the time series with fixed capcities (1991): {snr}"
)

### Contextualization of avoided losses 

In [ ]:
total_net_generation_ch = (
    (
        df_historical_data[df_historical_data["Jahr"] < 2023]
        .groupby("Jahr")
        .agg(sum)["Erzeugung_netto_GWh"]
    )
    * 1e-3
)

additional_generation = round(
    df_hydropower_yearly["Estimated Generation No Beta"]
    - df_hydropower_yearly["Estimated Generation No Beta_fixed_system_1991"],
    2,
)

print(
    f"""
      The implicit mitigation in RoR hydropower generation allowed to avoid {additional_generation.loc[2022]} TWh in losses,
      or {round(additional_generation.loc[2022] / total_net_generation_ch.loc[2022] * 100, 2)} % of the total net generation in Switzerland in 2022.
      """
)

# Supplementary material

## Sup Fig 1 - Run-of-river hydropower in Switzerland

In [ ]:
national_analysis.plot_ror_map_capacities_hist(
    save=True, output_filename="sup_fig_1.pdf", file_format="pdf"
)

In [ ]:
ror_capacity_ch = national_analysis.gdf_hydropower_locations.loc[
    (
        national_analysis.gdf_hydropower_locations["WASTANumber"].isin(
            national_analysis.ds_hydropower_generation.hydropower.values
        )
    ),
    "Capacity",
].sum()

print(
    f"The capacity of RoR hydropower in Switzerland is currently {ror_capacity_ch} GW"
)

In [ ]:
national_analysis.gdf_hydropower_locations.loc[
    (
        national_analysis.gdf_hydropower_locations["WASTANumber"].isin(
            national_analysis.ds_hydropower_generation.hydropower.values
        )
    ),
    "Capacity",
].describe()

In [ ]:
ror_capacity_90th_percentile = national_analysis.gdf_hydropower_locations.loc[
    (
        national_analysis.gdf_hydropower_locations["WASTANumber"].isin(
            national_analysis.ds_hydropower_generation.hydropower.values
        )
    ),
    "Capacity",
].quantile(0.9)

ror_capacity_fraction_90th_percentile_and_above = (
    national_analysis.gdf_hydropower_locations.loc[
        (
            national_analysis.gdf_hydropower_locations["WASTANumber"].isin(
                national_analysis.ds_hydropower_generation.hydropower.values
            )
        )
        & (
            national_analysis.gdf_hydropower_locations["Capacity"]
            >= ror_capacity_90th_percentile
        ),
        "Capacity",
    ].sum()
    / ror_capacity_ch
)

print(
    f"The RoR hydropower plants in Switzerland with capacities equal or above the 90th percentile constitute {round(ror_capacity_fraction_90th_percentile_and_above * 100, 2)}% of the total RoR capacity"
)

In [ ]:
national_analysis.gdf_hydropower_locations

## Sup Fig 2 - RoR hydropower high production years

The **total number of high production years of hydropower plants (quantile >= 0.9) per decade** is decreasing, even with an expanding infrastructure.

In [ ]:
national_analysis.plot_hist_prod_quantile_threshold_per_decade(
    yearly=True,
    variable_name="hp_no_beta",
    quantile_threshold=0.9,
    higher_than=True,
    with_operation_start=True,
    output_filename="sup_fig_2.pdf",
    save=True,
    file_format="pdf",
)

## Sup Fig 3 & 4 - RoR hydropower generation trends

In [ ]:
national_analysis.plot_trend_analysis_per_month(
    variable_name="hp_no_beta",
    save=True,
    output_filename="sup_fig_3.pdf",
    file_format="pdf",
)

In [ ]:
national_analysis.plot_winter_trend_map_and_distribution(
    variable_name="hp_no_beta_winter",
    save=True,
    output_filename="sup_fig_4.pdf",
    file_format="pdf",
)